# 1. Bibliotecas

In [1]:
import pandas as pd
import pyodbc
import pandasql as psql
import numpy as np

# 2. conexion integra

In [2]:
con = pyodbc.connect(dsn='Integra')

# 3. Parametros

In [3]:
Fecha_proceso=202403
UF=37601

# 4. Extraccion informacion

### 4.1 Información afiliados

In [5]:
Informacion_afiliados =pd.read_sql(f"""
WITH ExpuestosSegmento AS (
    SELECT
        cc.ID_FOLIO_CONTRATO,
        cc.ID_CORRELATIVO_PLAN,
        cc.F_VALOR_BASE_PLAN_UF AS F_VALOR_BASE_PLAN,
        cc.F_PRECIO_PLAN_UF AS F_PRECIO_PLAN,
        cc.F_RENTA_IMPONIBLE_AFILIADO,
        cc.F_MONTO_COTIZACION_PACTADA_UF AS F_MONTO_COTIZACION_PACTADA,
        cc.F_MONTO_COTIZ_PACTADA_GES,
        (cc.F_PRECIO_PLAN_UF + cc.F_MONTO_COTIZ_PACTADA_GES) AS F_TOTAL_PLAN_GES,
        cc.F_FACTOR_GRUPO_FAMILIAR,
        cc.F_NUMERO_DE_BENEFICIARIOS,
        cc.DS_COMUNA,
        pp.ID_CODIGO_COMERCIAL,
        pp.ID_TIPO_PLAN,
        (cc.F_MONTO_COTIZ_PACTADA_GES * 37800 + cc.F_VALOR_BASE_PLAN_UF * cc.F_FACTOR_GRUPO_FAMILIAR) AS pactado_plan_ges,
        clv.Tipificacion,
        clv.SIGNO
    FROM 
        integra.BT_CON_CONTRATO cc
    JOIN
        integra.BT_PRO_PRODUCTOS pp
    ON
        cc.ID_CORRELATIVO_PLAN = pp.ID_CORRELATIVO
        AND cc.ID_FECHA_PROCESO = pp.ID_FECHA_PROCESO
    LEFT JOIN
        DWH_CONSALUD.integra.GI_CLV clv
    ON
        cc.ID_FOLIO_CONTRATO = clv.Folio
        AND cc.ID_FECHA_PROCESO = clv.Fecha_Proceso
    WHERE 
        cc.ID_FECHA_PROCESO = {Fecha_proceso}
        AND cc.ID_VIGENCIA_SUSCRIPCION = 1 
        AND cc.ID_CODIGO_TIPO_AFILIADO <> 2
),
expseghosp AS (
    SELECT
        ExpuestosSegmento.*,
        COALESCE(SUM(DW_TARIF_HOSPITALARIO.F_NRO_SESIONES), 0) AS Suma_Monto_Prestacion_HOSPITALARIO,
        COALESCE(SUM(DW_TARIF_HOSPITALARIO.F_MONTO_ISAPRE), 0) AS Suma_Monto_Isapre_HOSPITALARIO,
        DW_TARIF_HOSPITALARIO.ID_RUT_PRESTADOR
    FROM
        ExpuestosSegmento
    LEFT JOIN
        DWH_CONSALUD.integra.DW_TARIF_HOSPITALARIO
    ON
        ExpuestosSegmento.ID_FOLIO_CONTRATO = DW_TARIF_HOSPITALARIO.ID_FOLIO_AFILIADO
    GROUP BY
        ExpuestosSegmento.ID_FOLIO_CONTRATO,
        ExpuestosSegmento.ID_CORRELATIVO_PLAN,
        ExpuestosSegmento.F_VALOR_BASE_PLAN,
        ExpuestosSegmento.F_PRECIO_PLAN,
        ExpuestosSegmento.F_RENTA_IMPONIBLE_AFILIADO,
        ExpuestosSegmento.F_MONTO_COTIZACION_PACTADA,
        ExpuestosSegmento.F_MONTO_COTIZ_PACTADA_GES,
        ExpuestosSegmento.F_FACTOR_GRUPO_FAMILIAR,
        ExpuestosSegmento.F_NUMERO_DE_BENEFICIARIOS,
        ExpuestosSegmento.ID_CODIGO_COMERCIAL,
        ExpuestosSegmento.ID_TIPO_PLAN,
        ExpuestosSegmento.pactado_plan_ges,
        DW_TARIF_HOSPITALARIO.ID_RUT_PRESTADOR,
        ExpuestosSegmento.Tipificacion,
        ExpuestosSegmento.SIGNO,
        ExpuestosSegmento.DS_COMUNA
),
top3 AS (
    SELECT 
        *,
        ROW_NUMBER() OVER (PARTITION BY ID_FOLIO_CONTRATO ORDER BY Suma_Monto_Prestacion_HOSPITALARIO DESC) AS Rank_RUT_Prestador
    FROM 
        expseghosp
)
SELECT 
    top3.ID_FOLIO_CONTRATO,
    top3.DS_COMUNA,
    (COALESCE(top3.Tipificacion, '0') || '-' || top3.SIGNO) AS Tipificacion_Signo,
    MAX(top3.F_VALOR_BASE_PLAN) AS F_VALOR_BASE_PLAN,
    MAX(top3.F_PRECIO_PLAN) AS F_PRECIO_PLAN,
    MAX(top3.F_RENTA_IMPONIBLE_AFILIADO) AS F_RENTA_IMPONIBLE_AFILIADO,
    MAX(top3.F_MONTO_COTIZACION_PACTADA) AS F_MONTO_COTIZACION_PACTADA,
    MAX(top3.F_MONTO_COTIZ_PACTADA_GES) AS F_MONTO_COTIZ_PACTADA_GES,
    MAX(top3.F_TOTAL_PLAN_GES) AS F_TOTAL_PLAN_GES,
    MAX(top3.F_FACTOR_GRUPO_FAMILIAR) AS F_FACTOR_GRUPO_FAMILIAR,
    MAX(top3.F_NUMERO_DE_BENEFICIARIOS) AS F_CANT_BENEFICIARIOS_BENEF,
    MAX(top3.ID_CODIGO_COMERCIAL) AS ID_CODIGO_COMERCIAL,
    MAX(CASE WHEN top3.Rank_RUT_Prestador = 1 THEN bt.DS_NOMBRE_FANTASIA END) AS preferido1_hosp,
    MAX(CASE WHEN top3.Rank_RUT_Prestador = 2 THEN bt.DS_NOMBRE_FANTASIA END) AS preferido2_hosp,
    MAX(CASE WHEN top3.Rank_RUT_Prestador = 3 THEN bt.DS_NOMBRE_FANTASIA END) AS preferido3_hosp
FROM 
    top3
LEFT JOIN 
   integra.LK_PRO_GRUPO_CONVENIOS AS bt ON top3.ID_RUT_PRESTADOR = bt.ID_RUT_PRESTADOR
WHERE 
    top3.Rank_RUT_Prestador <= 3
GROUP BY 
    top3.ID_FOLIO_CONTRATO, 
    top3.Tipificacion,
    top3.SIGNO,
    top3.DS_COMUNA,
    top3.ID_TIPO_PLAN
""",con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\1876507043.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  Informacion_afiliados =pd.read_sql(f"""


### 4.2 Factor de riesgo 343

In [6]:
factor_riesgo_343=pd.read_sql(f"""
SELECT
    c.ID_FOLIO_CONTRATO,
    SUM(fr.FR) AS Suma_FR
FROM 
    DWH_CONSALUD.integra.BT_CON_BENEFCONTRATO c
LEFT JOIN 
    integra.DW_GI_DICCIONARIO_FOLIO d
ON 
    c.ID_FOLIO_CONTRATO = d.FOLIO 
LEFT JOIN 
    (SELECT 'COTI' AS TIPO_COTIZANTE, '0-2A' AS RANGO_EDAD, 0 AS EDAD, 0 AS FR UNION ALL
     SELECT 'COTI', '2-5A', 2, 0.6 UNION ALL
     SELECT 'COTI', '5-10A', 5, 0.6 UNION ALL
     SELECT 'COTI', '10-15A', 10, 0.6 UNION ALL
     SELECT 'COTI', '15-20A', 15, 0.6 UNION ALL
     SELECT 'COTI', '20-25A', 20, 0.9 UNION ALL
     SELECT 'COTI', '25-30A', 25, 1.0 UNION ALL
     SELECT 'COTI', '30-35A', 30, 1.0 UNION ALL
     SELECT 'COTI', '35-40A', 35, 1.3 UNION ALL
     SELECT 'COTI', '40-45A', 40, 1.3 UNION ALL
     SELECT 'COTI', '45-50A', 45, 1.4 UNION ALL
     SELECT 'COTI', '50-55A', 50, 1.4 UNION ALL
     SELECT 'COTI', '55-60A', 55, 2.0 UNION ALL
     SELECT 'COTI', '60-65A', 60, 2.0 UNION ALL
     SELECT 'COTI', '65-70A', 65, 2.4 UNION ALL
     SELECT 'COTI', '70-75A', 70, 2.4 UNION ALL
     SELECT 'COTI', '75-80A', 75, 2.4 UNION ALL
     SELECT 'COTI', '>=80', 80, 2.4 UNION ALL
     SELECT 'CARGA', '0-2A', 0, 0 UNION ALL
     SELECT 'CARGA', '2-5A', 2, 0.6 UNION ALL
     SELECT 'CARGA', '5-10A', 5, 0.6 UNION ALL
     SELECT 'CARGA', '10-15A', 10, 0.6 UNION ALL
     SELECT 'CARGA', '15-20A', 15, 0.6 UNION ALL
     SELECT 'CARGA', '20-25A', 20, 0.7 UNION ALL
     SELECT 'CARGA', '25-30A', 25, 0.7 UNION ALL
     SELECT 'CARGA', '30-35A', 30, 0.7 UNION ALL
     SELECT 'CARGA', '35-40A', 35, 0.9 UNION ALL
     SELECT 'CARGA', '40-45A', 40, 0.9 UNION ALL
     SELECT 'CARGA', '45-50A', 45, 1.0 UNION ALL
     SELECT 'CARGA', '50-55A', 50, 1.0 UNION ALL
     SELECT 'CARGA', '55-60A', 55, 1.4 UNION ALL
     SELECT 'CARGA', '60-65A', 60, 1.4 UNION ALL
     SELECT 'CARGA', '65-70A', 65, 2.2 UNION ALL
     SELECT 'CARGA', '70-75A', 70, 2.2 UNION ALL
     SELECT 'CARGA', '75-80A', 75, 2.2 UNION ALL
     SELECT 'CARGA', '>=80', 80, 2.2) fr
ON 
    CASE 
        WHEN c.ID_CODIGO_CARGA = 0 THEN 'COTI'
        ELSE 'CARGA'
    END = fr.TIPO_COTIZANTE
    AND 
    CASE 
        WHEN c.ID_EDAD_BENEF >= 0 AND c.ID_EDAD_BENEF < 2 THEN '0-2A'
        WHEN c.ID_EDAD_BENEF >= 2 AND c.ID_EDAD_BENEF < 5 THEN '2-5A'
        WHEN c.ID_EDAD_BENEF >= 5 AND c.ID_EDAD_BENEF < 10 THEN '5-10A'
        WHEN c.ID_EDAD_BENEF >= 10 AND c.ID_EDAD_BENEF < 15 THEN '10-15A'
        WHEN c.ID_EDAD_BENEF >= 15 AND c.ID_EDAD_BENEF < 20 THEN '15-20A'
        WHEN c.ID_EDAD_BENEF >= 20 AND c.ID_EDAD_BENEF < 25 THEN '20-25A'
        WHEN c.ID_EDAD_BENEF >= 25 AND c.ID_EDAD_BENEF < 30 THEN '25-30A'
        WHEN c.ID_EDAD_BENEF >= 30 AND c.ID_EDAD_BENEF < 35 THEN '30-35A'
        WHEN c.ID_EDAD_BENEF >= 35 AND c.ID_EDAD_BENEF < 40 THEN '35-40A'
        WHEN c.ID_EDAD_BENEF >= 40 AND c.ID_EDAD_BENEF < 45 THEN '40-45A'
        WHEN c.ID_EDAD_BENEF >= 45 AND c.ID_EDAD_BENEF < 50 THEN '45-50A'
        WHEN c.ID_EDAD_BENEF >= 50 AND c.ID_EDAD_BENEF < 55 THEN '50-55A'
        WHEN c.ID_EDAD_BENEF >= 55 AND c.ID_EDAD_BENEF < 60 THEN '55-60A'
        WHEN c.ID_EDAD_BENEF >= 60 AND c.ID_EDAD_BENEF < 65 THEN '60-65A'
        WHEN c.ID_EDAD_BENEF >= 65 AND c.ID_EDAD_BENEF < 70 THEN '65-70A'
        WHEN c.ID_EDAD_BENEF >= 70 AND c.ID_EDAD_BENEF < 75 THEN '70-75A'
        WHEN c.ID_EDAD_BENEF >= 75 AND c.ID_EDAD_BENEF < 80 THEN '75-80A'
        WHEN c.ID_EDAD_BENEF >= 80 THEN '>=80'
        ELSE 'Sin rango'
    END = fr.RANGO_EDAD
WHERE 
    ID_VIGENCIA_SUSCRIPCION = 1 
    AND ID_FECHA_PROCESO = {Fecha_proceso}
GROUP BY 
    c.ID_FOLIO_CONTRATO
""",con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\1213219486.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  factor_riesgo_343=pd.read_sql(f"""


In [7]:
Informacion_afiliados=psql.sqldf("""
SELECT 
    a.*,
    b.Suma_FR AS Factor_riesgo_343
FROM Informacion_afiliados a
LEFT JOIN factor_riesgo_343 b
ON a.ID_FOLIO_CONTRATO=b.ID_FOLIO_CONTRATO
""")

### 4.3 Diccionario hash_folio

In [8]:
diccionario_hash_folio=pd.read_sql("""SELECT d.* 
    FROM DWH_CONSALUD.integra.DW_GI_DICCIONARIO_FOLIO d
    LEFT JOIN integra.BT_CON_CONTRATO cc
ON cc.ID_FOLIO_CONTRATO=d.FOLIO
WHERE 
	cc.ID_FECHA_PROCESO = 202403
	AND cc.ID_VIGENCIA_SUSCRIPCION = 1 
	AND cc.ID_CODIGO_TIPO_AFILIADO <> 2""", con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\2776950566.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  diccionario_hash_folio=pd.read_sql("""SELECT d.*


### 4.4 Diccionarios convenios

In [9]:
diccionario_convenios=pd.read_sql("""
SELECT 
    ID_RUT_PRESTADOR,
    DS_NOMBRE_FANTASIA,
    DS_GRUPO 
FROM integra.LK_PRO_GRUPO_CONVENIOS""", con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\2700304122.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  diccionario_convenios=pd.read_sql("""


### 4.5 Prestador preferido ambulatorio

In [10]:
# Apertura prestadores preferidos ambulatorios - consultas
ranking_amb_consulta = pd.read_csv("C:/Users/cgarciaa/Desktop/Proyecto competitividad-scoring/Prestador preferido/202403/Prestador preferido amb consultas 202403.csv", sep=";")
ranking_amb_consulta = pd.merge(ranking_amb_consulta, diccionario_hash_folio, left_on='folio', right_on='HASH_FOLIO', how='left')
ranking_amb_consulta = pd.merge(ranking_amb_consulta, diccionario_convenios, left_on='rut_prestador', right_on='ID_RUT_PRESTADOR', how='left')
ranking_amb_consulta = ranking_amb_consulta.dropna(subset=['ID_RUT_PRESTADOR'])
ranking_amb_consulta['DS_GRUPO'] = pd.Categorical(ranking_amb_consulta['DS_GRUPO'], categories=['G1', 'G2', 'G3', 'G4', 'G5', 'G6'], ordered=True)
ranking_amb_consulta = ranking_amb_consulta.sort_values(by=['FOLIO', 'DS_GRUPO'])
ranking_amb_consulta = ranking_amb_consulta.drop(columns=['folio'])
ranking_amb_consulta = ranking_amb_consulta.drop_duplicates(subset=['FOLIO'])
ranking_amb_consulta = ranking_amb_consulta[['FOLIO', 'DS_NOMBRE_FANTASIA']]
# Apertura prestadores preferidos - examenes
ranking_amb_examenes = pd.read_csv("C:/Users/cgarciaa/Desktop/Proyecto competitividad-scoring/Prestador preferido/202403/Prestador preferido amb examenes 202403.csv", sep=";")
ranking_amb_examenes = pd.merge(ranking_amb_examenes, diccionario_hash_folio, left_on='folio', right_on='HASH_FOLIO', how='left')
ranking_amb_examenes = pd.merge(ranking_amb_examenes, diccionario_convenios, left_on='rut_prestador', right_on='ID_RUT_PRESTADOR', how='left')
ranking_amb_examenes = ranking_amb_examenes.dropna(subset=['ID_RUT_PRESTADOR'])
ranking_amb_examenes['DS_GRUPO'] = pd.Categorical(ranking_amb_examenes['DS_GRUPO'], categories=['G1', 'G2', 'G3', 'G4', 'G5', 'G6'], ordered=True)
ranking_amb_examenes = ranking_amb_examenes.sort_values(by=['FOLIO', 'DS_GRUPO'])
ranking_amb_examenes = ranking_amb_examenes.drop(columns=['folio'])
ranking_amb_examenes = ranking_amb_examenes.drop_duplicates(subset=['FOLIO'])

# Apertura prestadores preferidos - imagenología
ranking_amb_imagenologia = pd.read_csv("C:/Users/cgarciaa/Desktop/Proyecto competitividad-scoring/Prestador preferido/202403/Prestador preferido amb imagenologia 202403.csv", sep=";")
ranking_amb_imagenologia = pd.merge(ranking_amb_imagenologia, diccionario_hash_folio, left_on='folio', right_on='HASH_FOLIO', how='left')
ranking_amb_imagenologia = pd.merge(ranking_amb_imagenologia, diccionario_convenios, left_on='rut_prestador', right_on='ID_RUT_PRESTADOR', how='left')
ranking_amb_imagenologia = ranking_amb_imagenologia.dropna(subset=['ID_RUT_PRESTADOR'])
ranking_amb_imagenologia['DS_GRUPO'] = pd.Categorical(ranking_amb_imagenologia['DS_GRUPO'], categories=['G1', 'G2', 'G3', 'G4', 'G5', 'G6'], ordered=True)
ranking_amb_imagenologia = ranking_amb_imagenologia.sort_values(by=['FOLIO', 'DS_GRUPO'])
ranking_amb_imagenologia = ranking_amb_imagenologia.drop(columns=['folio'])
ranking_amb_imagenologia = ranking_amb_imagenologia.drop_duplicates(subset=['FOLIO'])

In [11]:
# Función para agregar prestador preferido
def agregacion_prestador_preferido(Informacion_afiliados, ranking_prestador_preferido, tipo_prestador):
    query = f"""
    SELECT 
        a.*,
        b.DS_NOMBRE_FANTASIA as {tipo_prestador}
    FROM Informacion_afiliados a
    LEFT JOIN ranking_prestador_preferido b
    ON a.ID_FOLIO_CONTRATO = b.FOLIO
    """
    return psql.sqldf(query, locals())
# Agregar prestador preferido de consultas
Informacion_afiliados = agregacion_prestador_preferido(Informacion_afiliados, ranking_amb_consulta, 'preferido_consulta')
# Agregar prestador preferido de exámenes
Informacion_afiliados = agregacion_prestador_preferido(Informacion_afiliados, ranking_amb_examenes, 'preferido_examenes')
# Agregar prestador preferido de imagenología
Informacion_afiliados = agregacion_prestador_preferido(Informacion_afiliados, ranking_amb_imagenologia, 'preferido_imagenologia')

### 4.6 score total planes stock

In [12]:
score_total_planes_stock=pd.read_sql("""
SELECT ID_CODIGO_COMERCIAL,
		F_SCORE_HOSP
FROM DWH_CONSALUD.integra.DW_PRO_SCORE_DETALLE
GROUP BY ID_CODIGO_COMERCIAL,
		  F_SCORE_HOSP""", con)

Informacion_afiliados['ID_CODIGO_COMERCIAL'] = Informacion_afiliados['ID_CODIGO_COMERCIAL'].str.strip()

Informacion_afiliados=psql.sqldf("""
SELECT a.*,
       b.F_SCORE_HOSP AS score_total_plan_actual
FROM Informacion_afiliados a
LEFT JOIN score_total_planes_stock b
ON a.ID_CODIGO_COMERCIAL=b.ID_CODIGO_COMERCIAL""")

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\3766393283.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  score_total_planes_stock=pd.read_sql("""


### 4.7 score prestador planes stock

In [13]:
score_prestador_planes_stock=pd.read_sql("""
    SELECT 
        ID_CODIGO_COMERCIAL,
        DS_NOMBRE_FANTASIA,
        F_SCORE_PRESTADOR_HOSP,
        F_SCORE_HOSP
    FROM DWH_CONSALUD.integra.DW_PRO_SCORE_DETALLE
    WHERE DS_NOMBRE_FANTASIA != '' AND
          F_SCORE_PRESTADOR_HOSP != 0
    GROUP BY 
        ID_CODIGO_COMERCIAL,
        DS_NOMBRE_FANTASIA,
        F_SCORE_PRESTADOR_HOSP,
        F_SCORE_HOSP""", con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\2096121293.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  score_prestador_planes_stock=pd.read_sql("""


In [14]:
def calcula_score_total(prestador_preferido, nombre_columna_valor_score):
    query = f"""
        SELECT
            a.*,
            b.F_SCORE_PRESTADOR_HOSP AS {nombre_columna_valor_score}
        FROM 
            Informacion_afiliados a
        LEFT JOIN
            score_prestador_planes_stock b
        ON a.{prestador_preferido} = b.DS_NOMBRE_FANTASIA AND
           a.ID_CODIGO_COMERCIAL = b.ID_CODIGO_COMERCIAL
    """
    consulta = psql.sqldf(query, globals())
    return consulta

# Llamar a la función
Informacion_afiliados = calcula_score_total('preferido1_hosp', 'score_pref1')
Informacion_afiliados = calcula_score_total('preferido2_hosp', 'score_pref2')
Informacion_afiliados = calcula_score_total('preferido3_hosp', 'score_pref3')
Informacion_afiliados = calcula_score_total('preferido_consulta', 'score_pref_consulta')
Informacion_afiliados = calcula_score_total('preferido_examenes', 'score_pref_examenes')
Informacion_afiliados = calcula_score_total('preferido_imagenologia', 'score_pref_imagenologia')

### 4.8 Diccionario zona

In [15]:
diccionario_zona=pd.read_csv("C:/Users/cgarciaa/Desktop/Proyecto competitividad-scoring/Diccionarios/diccionario sucursal.csv", sep=';')
Informacion_afiliados['DS_COMUNA'] = Informacion_afiliados['DS_COMUNA'].str.rstrip()
Informacion_afiliados=psql.sqldf("""
SELECT 
    a.*,
    b.Ds_Zona
FROM Informacion_afiliados a
LEFT JOIN diccionario_zona b
on a.DS_COMUNA=b.Ds_comuna
""")

### 4.9 Pactado simulacion 202403

In [16]:
pactado_simulacion_202403=pd.read_csv("C:/Users/cgarciaa/Desktop/Pactado_simulacion_202403.csv", sep=';')

Informacion_afiliados=psql.sqldf("""
    SELECT 
        a.*,
        b.Pactado_simulacion
    FROM
        Informacion_afiliados a
    LEFT JOIN 
        pactado_simulacion_202403 b
    ON 
        a.ID_FOLIO_CONTRATO=b.FOLIO
""")

### 4.10 Tipificacion gasto clv

In [17]:
gasto_clv=pd.read_csv("C:/Users/cgarciaa/Desktop/clv_tipificacion.csv", sep=',')
gasto_clv=psql.sqldf("""
    SELECT b.FOLIO,
           CONCAT(a.tipificacion, '-', a.signo) AS tipificacion_signo
    FROM gasto_clv a
    LEFT JOIN diccionario_hash_folio b
    ON a.hash_folio=b.HASH_FOLIO""")
Informacion_afiliados = Informacion_afiliados.drop(columns=['Tipificacion_Signo'])
Informacion_afiliados=psql.sqldf("""
    SELECT a.*,
           b.tipificacion_signo
    FROM Informacion_afiliados a
    LEFT JOIN gasto_clv b
    ON a.ID_FOLIO_CONTRATO=b.FOLIO""")

### 4.11 Marca retenible

In [18]:
marca_retenible=pd.read_csv("C:/Users/cgarciaa/Desktop/CARTERA_JUNIO_CON_FILTROS_RETENCION_MARCAS_ABIERTAS.csv", sep=';')
Informacion_afiliados=psql.sqldf("""
    SELECT a.*,
           b.RETENIBLE
    FROM Informacion_afiliados a
    LEFT JOIN marca_retenible b
    ON a.ID_FOLIO_CONTRATO=b.ID_FOLIO_CONTRATO""")
Informacion_afiliados['RETENIBLE'] = Informacion_afiliados['RETENIBLE'].fillna(0)

### 4.12 Score total planes comercializables

In [19]:
lista_comercializables = pd.read_csv("C:/Users/cgarciaa/Desktop/planes comercializables julio 2024.csv", sep=';', encoding='latin-1')
lista_comercializables.rename(columns={'CÓDIGO COMERCIAL': 'codigo_comercial'}, inplace=True)
lista_comercializables.rename(columns={'VALOR BASE': 'Valor_base'}, inplace=True)
codigo_comercial_list = lista_comercializables['codigo_comercial'].tolist()

# Convertir la lista de códigos comerciales en un formato adecuado para la consulta SQL
codigo_comercial_list_str = tuple(codigo_comercial_list)

score_total_planes_comercializables_consalud = psql.sqldf(f"""
    SELECT 
        a.*,
        b.Valor_base,
        b.ZONA,
        CASE 
            WHEN a.ID_CODIGO_COMERCIAL LIKE '13-%' THEN 'preferente'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '14-%' THEN 'cerrado'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '15-%' THEN 'libre eleccion'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '63-%' THEN 'empresas'
            ELSE 'otro'
        END AS tipo_plan,
        CASE 
            WHEN SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 3) = 'RET' THEN 'retencion'
            WHEN SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 4) = 'MANT' THEN 'mantencion'
            ELSE 'normal'
        END AS marca_ret_mant
    FROM score_total_planes_stock a
    LEFT JOIN lista_comercializables b
    ON a.ID_CODIGO_COMERCIAL = b.codigo_comercial
    WHERE a.ID_CODIGO_COMERCIAL in {codigo_comercial_list_str}
      AND a.ID_CODIGO_COMERCIAL NOT LIKE '63-%'
      AND SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 4) != 'MANT'
""", globals())

In [20]:
data = {
    'ID_CODIGO_COMERCIAL': ['EO1', 'nuevo_ret_s', 'nuevo_ret_c', 'nuevo_ret_n', 'nuevo_ret_o', 'nuevo_ret_sur','EO05', 'nuevo_ret_2_s', 'nuevo_ret_2_c', 'nuevo_ret_2_n', 'nuevo_ret_2_o', 'nuevo_ret_2_sur', 'N1'],
    'F_SCORE_HOSP': [2.47, 2.06, 2.06, 2.06, 2.06, 2.06, 
                     2.47, 2.58, 2.58, 2.58, 2.58, 2.58, 3.28],
    'Valor_base': [1.64, 1.34, 1.34, 1.34, 1.34, 1.34, 1.51, 1.45, 1.45, 1.45, 1.45, 1.45, 1.94],
    'ZONA': ['SANTIAGO', 'SANTIAGO', 'CENTRO', 'NORTE', 'OCTAVA', 'SUR', 
             'SANTIAGO', 'SANTIAGO', 'CENTRO', 'NORTE', 'OCTAVA', 'SUR', 'NORTE'],
    'tipo_plan': ['preferente', 'preferente', 'preferente', 'preferente', 'preferente', 'preferente', 
                  'preferente', 'preferente', 'preferente', 'preferente', 'preferente', 'preferente', 'preferente'],
    'marca_ret_mant': ['normal', 'retencion', 'retencion', 'retencion', 'retencion', 'retencion',
                       'normal', 'retencion', 'retencion', 'retencion', 'retencion', 'retencion', 'normal']
}
score_total_planes_comercializables_consalud_adicionales = pd.DataFrame(data)
score_total_planes_comercializables_consalud = pd.concat([score_total_planes_comercializables_consalud, score_total_planes_comercializables_consalud_adicionales], axis=0)
score_total_planes_comercializables_consalud.reset_index(drop=True, inplace=True)

### 4.13 Score total planes comercializables competencia

In [21]:
score_total_planes_comercializables_competencia = pd.read_sql("""
    SELECT 
        replace(ID_CODIGO_COMERCIAL,'Â','') AS ID_CODIGO_COMERCIAL,
        F_SCORE_HOSP_PLAN,
        F_GES,
        F_VALOR_BASE,
        DS_ZONA_COMERCIALIZACION,
        DS_ISAPRE
    FROM 
        DWH_CONSALUD.integra.DW_PRO_SCOR_INDUSTRIA
    WHERE 
        ID_FECHA_TERMINO_COMERCIALIZACION = 0 
        AND ID_CODIGO_COMERCIAL NOT IN ('BFDC230802', 'BFDC230803', 'BFDC230812', 'BFDC230822', 'BFDC230813', 'BFDC230823', 'BFDC230833')
    GROUP BY 
        ID_CODIGO_COMERCIAL,
        F_SCORE_HOSP_PLAN,
        F_GES,
        F_VALOR_BASE,
        DS_ZONA_COMERCIALIZACION,
        DS_ISAPRE
""", con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\1676729455.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  score_total_planes_comercializables_competencia = pd.read_sql("""


### 4.14 Informacion afiliados resumida

In [22]:
salida_afiliados=psql.sqldf("""
SELECT
    ID_FOLIO_CONTRATO,
    tipificacion_signo,
    Ds_Zona,
    F_VALOR_BASE_PLAN,
    F_RENTA_IMPONIBLE_AFILIADO,
    F_CANT_BENEFICIARIOS_BENEF,
    ID_CODIGO_COMERCIAL,
    Score_total_plan_actual,
    Pactado_simulacion,
    RETENIBLE
FROM Informacion_afiliados
""")

### 4.15 Score prestador planes comercilizables consalud

In [23]:
score_prestador_planes_comercializables_consalud = psql.sqldf(f"""
    SELECT 
        a.*,
        b.Valor_base,
        b.ZONA,
        b.tipo_plan_cobertura,
        CASE 
            WHEN a.ID_CODIGO_COMERCIAL LIKE '13-%' THEN 'preferente'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '14-%' THEN 'cerrado'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '15-%' THEN 'libre eleccion'
            WHEN a.ID_CODIGO_COMERCIAL LIKE '63-%' THEN 'empresas'
            ELSE 'otro'
        END AS tipo_plan,
        CASE 
            WHEN SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 3) = 'RET' THEN 'retencion'
            WHEN SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 4) = 'MANT' THEN 'mantencion'
            ELSE 'normal'
        END AS marca_ret_mant
    FROM score_prestador_planes_stock a
    LEFT JOIN lista_comercializables b
    ON a.ID_CODIGO_COMERCIAL = b.codigo_comercial
    WHERE a.ID_CODIGO_COMERCIAL in {codigo_comercial_list_str}
      AND a.ID_CODIGO_COMERCIAL NOT LIKE '63-%'
      AND SUBSTR(a.ID_CODIGO_COMERCIAL, 4, 4) != 'MANT'
""", globals())

score_planes_agregados=pd.read_csv("C:/Users/cgarciaa/Desktop/score_prestador_planes_agregados.csv", sep=';')
score_prestador_planes_comercializables_consalud = pd.concat([score_prestador_planes_comercializables_consalud, score_planes_agregados], axis=0)
score_prestador_planes_comercializables_consalud.reset_index(drop=True, inplace=True)

### 4.16 Score prestador planes comercializables competenica

In [24]:
score_prestador_planes_comercializables_competencia = pd.read_sql("""
    SELECT 
        ID_CODIGO_COMERCIAL,
        DS_NOMBRE_FANTASIA,
        F_SCORE_HOSP_PRESTADOR,
        F_SCORE_HOSP_PLAN,
        F_GES,
        F_VALOR_BASE,
        DS_ZONA_COMERCIALIZACION,
        DS_ISAPRE
    FROM 
        DWH_CONSALUD.integra.DW_PRO_SCOR_INDUSTRIA
    WHERE 
        ID_FECHA_TERMINO_COMERCIALIZACION = 0 
        AND ID_CODIGO_COMERCIAL NOT IN ('BFDC230802', 'BFDC230803', 'BFDC230812', 'BFDC230822', 'BFDC230813', 'BFDC230823', 'BFDC230833')
    GROUP BY 
        ID_CODIGO_COMERCIAL,
        DS_NOMBRE_FANTASIA,
        F_SCORE_HOSP_PRESTADOR,
        F_SCORE_HOSP_PLAN,
        F_GES,
        F_VALOR_BASE,
        DS_ZONA_COMERCIALIZACION,
        DS_ISAPRE
""", con)

C:\Users\cgarciaa\AppData\Local\Temp\ipykernel_18372\3062620254.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  score_prestador_planes_comercializables_competencia = pd.read_sql("""


### 4.17 Beneficio adicional

In [25]:
beneficio_adicional=pd.read_csv("C:/Users/cgarciaa/Desktop/Beneficio_Adicional_202403.csv", sep=";")
Informacion_afiliados=psql.sqldf("""
    SELECT 
        a.*,
        b.PRECIO_BENEFICIO_ADICIONAL
    FROM Informacion_afiliados a
    LEFT JOIN beneficio_adicional b
    ON a.ID_FOLIO_CONTRATO=b.FOLIO
""")
Informacion_afiliados['PRECIO_BENEFICIO_ADICIONAL'] = Informacion_afiliados['PRECIO_BENEFICIO_ADICIONAL'].fillna(0)

### 4.18 Seleccion de prestador preferido unico

In [26]:
Informacion_afiliados['prestador_preferido'] = Informacion_afiliados[['preferido1_hosp', 'preferido2_hosp', 'preferido3_hosp',
                                'preferido_consulta', 'preferido_examenes', 'preferido_imagenologia']].bfill(axis=1).iloc[:, 0]
Informacion_afiliados['score_preferido'] = np.select(
    [
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido1_hosp'],
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido2_hosp'],
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido3_hosp'],
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido_consulta'],
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido_examenes'],
        Informacion_afiliados['prestador_preferido'] == Informacion_afiliados['preferido_imagenologia']
    ],
    [
        Informacion_afiliados['score_pref1'],
        Informacion_afiliados['score_pref2'],
        Informacion_afiliados['score_pref3'],
        Informacion_afiliados['score_pref_consulta'],
        Informacion_afiliados['score_pref_examenes'],
        Informacion_afiliados['score_pref_imagenologia']
    ],
    default=np.nan
)
Informacion_afiliados.drop(columns=['preferido1_hosp', 'preferido2_hosp', 'preferido3_hosp',
                 'preferido_consulta', 'preferido_examenes', 'preferido_imagenologia',
                 'score_pref1', 'score_pref2', 'score_pref3',
                 'score_pref_consulta', 'score_pref_examenes', 'score_pref_imagenologia'], inplace=True)

# 5. Busqueda por score total

### 5.1 Busqueda por score total CONSALUD

In [27]:
Busqueda_por_score_total_consalud = psql.sqldf(f"""
SELECT 
    a.ID_FOLIO_CONTRATO,
    a.tipificacion_signo,
    a.F_VALOR_BASE_PLAN,
        CASE 
        WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
        ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
    END AS F_RENTA_IMPONIBLE_AFILIADO,
    a.ID_CODIGO_COMERCIAL,
    a.Factor_riesgo_343,
    a.Ds_Zona,
    a.Pactado_simulacion,
    a.score_total_plan_actual,
    a.F_CANT_BENEFICIARIOS_BENEF,
    a.F_MONTO_COTIZ_PACTADA_GES,
    a.RETENIBLE,
    a.PRECIO_BENEFICIO_ADICIONAL,
    b.ID_CODIGO_COMERCIAL as opciones_planes,
    b.Valor_base AS valor_base_nuevas_opciones,
    b.F_SCORE_HOSP AS score_total_nuevas_opciones,
    b.tipo_plan,
    b.marca_ret_mant,
    (a.Factor_riesgo_343 * b.Valor_base + 0.602 * a.F_CANT_BENEFICIARIOS_BENEF) AS pactado_nuevas_opciones,
    ROUND((a.Factor_riesgo_343 * b.Valor_base + 0.602 * a.F_CANT_BENEFICIARIOS_BENEF) / 
        (CAST(CASE 
                WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
                ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
              END AS FLOAT) / {UF}), 4) as pactado_renta_nuevo
FROM
    Informacion_afiliados a
JOIN 
    score_total_planes_comercializables_consalud b
ON
    a.Ds_Zona = b.ZONA
WHERE
   NOT (a.RETENIBLE = 0 AND b.marca_ret_mant = 'retencion')
   AND   ROUND((a.Factor_riesgo_343 * b.Valor_base + 0.602 * a.F_RENTA_IMPONIBLE_AFILIADO) / 
       (CAST(CASE 
               WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
               ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
             END AS FLOAT) / {UF}), 4) >= 0.06
""")

In [28]:
Busqueda_por_score_total_consalud['Diferencia_score_total']=Busqueda_por_score_total_consalud['score_total_nuevas_opciones']-Busqueda_por_score_total_consalud['score_total_plan_actual']
Busqueda_por_score_total_consalud['Diferencia_pactado']=Busqueda_por_score_total_consalud['pactado_nuevas_opciones']-Busqueda_por_score_total_consalud['Pactado_simulacion']

In [29]:
Busqueda_por_score_total_consalud_filtado=psql.sqldf("""
    SELECT 
        *
    FROM Busqueda_por_score_total_consalud
    WHERE Diferencia_pactado < 0""")

In [30]:
Busqueda_por_score_total_consalud_filtado['busqueda'] = np.where((Busqueda_por_score_total_consalud_filtado['Diferencia_score_total'] > -0.2) & (Busqueda_por_score_total_consalud_filtado['Diferencia_score_total'] <= 0.2), 'busqueda 1',
                  np.where(Busqueda_por_score_total_consalud_filtado['Diferencia_score_total'] > 0.2, 'busqueda 2', 'busqueda 3'))

In [32]:
 df = Busqueda_por_score_total_consalud_filtado

# Función para obtener las métricas por tipo de búsqueda
def get_busqueda_metrics(df, busqueda_tipo):
    filtered_df = df[df['busqueda'] == busqueda_tipo]

    # Obtener cantidad de ofertas por ID_FOLIO_CONTRATO
    count_df = filtered_df.groupby('ID_FOLIO_CONTRATO').size().reset_index(name=f'cantidad_oferta_{busqueda_tipo}')

    # Obtener el pactado mínimo y sus correspondientes código comercial y score
    min_df = filtered_df.loc[filtered_df.groupby('ID_FOLIO_CONTRATO')['pactado_nuevas_opciones'].idxmin()]

    min_df = min_df[['ID_FOLIO_CONTRATO', 'pactado_nuevas_opciones', 'opciones_planes', 'Diferencia_score_total']]

    min_df.columns = [
        'ID_FOLIO_CONTRATO',
        f'pactado_minimo_{busqueda_tipo}',
        f'codigo_comercial_minimo_{busqueda_tipo}',
        f'score_minimo_{busqueda_tipo}'
    ]

    # Unir las dos tablas
    result_df = count_df.merge(min_df, on='ID_FOLIO_CONTRATO', how='left')

    return result_df

# Obtener métricas para cada tipo de búsqueda
busqueda_1_metrics = get_busqueda_metrics(df, 'busqueda 1')
busqueda_2_metrics = get_busqueda_metrics(df, 'busqueda 2')
busqueda_3_metrics = get_busqueda_metrics(df, 'busqueda 3')

# Combinar los resultados para cada tipo de búsqueda en un único DataFrame
Busqueda_por_score_total_consalud = busqueda_1_metrics.merge(busqueda_2_metrics, on='ID_FOLIO_CONTRATO', how='outer')\
                           .merge(busqueda_3_metrics, on='ID_FOLIO_CONTRATO', how='outer')

In [33]:
Busqueda_por_score_total_consalud = Busqueda_por_score_total_consalud.fillna("Sin Oferta")

### 5.2 Busqueda por score total COMPETENCIA

In [28]:
Busqueda_por_score_total_competencia = psql.sqldf(f"""
SELECT 
    a.ID_FOLIO_CONTRATO,
    a.Tipificacion_Signo,
    a.F_VALOR_BASE_PLAN,
    CASE 
        WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
        ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
    END AS F_RENTA_IMPONIBLE_AFILIADO,
    a.ID_CODIGO_COMERCIAL,
    a.Factor_riesgo_343,
    a.Ds_Zona,
    a.Pactado_simulacion,
    a.score_total_plan_actual,
    a.F_CANT_BENEFICIARIOS_BENEF,
    a.F_MONTO_COTIZ_PACTADA_GES,
    b.ID_CODIGO_COMERCIAL as opciones_planes,
    b.F_VALOR_BASE AS valor_base_nuevas_opciones,
    b.F_SCORE_HOSP_PLAN AS score_total_nuevas_opciones,
    b.DS_ISAPRE,
    b.F_GES AS Ges_Isapre,
    (a.Factor_riesgo_343*b.F_VALOR_BASE+b.F_GES*a.F_CANT_BENEFICIARIOS_BENEF) as pactado_nuevas_opciones,
    ROUND((a.Factor_riesgo_343 * b.F_VALOR_BASE + b.F_GES * a.F_CANT_BENEFICIARIOS_BENEF) / 
        (CAST(CASE 
                WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
                ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
              END AS FLOAT) / {UF}), 4) as pactado_renta_nuevo
FROM
    Informacion_afiliados a
JOIN 
    score_total_planes_comercializables_competencia b
ON
    a.Ds_Zona = b.DS_ZONA_COMERCIALIZACION
WHERE
    ROUND((a.Factor_riesgo_343 * b.F_VALOR_BASE + 0.602 * a.F_RENTA_IMPONIBLE_AFILIADO) / 
       (CAST(CASE 
               WHEN COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500) > 3096360 THEN 3096360
               ELSE COALESCE(NULLIF(a.F_RENTA_IMPONIBLE_AFILIADO, 0), 36500)
             END AS FLOAT) / {UF}), 4) >= 0.06
    AND ABS(a.Score_total_plan_actual - b.F_SCORE_HOSP_PLAN) <= 1.0 
""")

In [4]:
Busqueda_por_score_total_competencia

NameError: name 'Busqueda_por_score_total_competencia' is not defined